In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [982 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,515 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-13 01:06:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.58MB/s    in 0.2s    

2022-06-13 01:06:50 (5.58 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
#df=spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
#df.show(5)


In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16121565|             5|
|   48146680|             1|
|   11062912|             1|
|   51451778|             1|
|   27920838|             1|
|    4919528|             3|
|   47802851|             2|
|   42560427|             1|
|   29467780|             1|
|   49703087|            12|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show(10)


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CJL5ES|L.B. Evans Men's ...|
|B002MCVHUQ|New Balance Insol...|
|B00UXBFW6S|Kenox Sling Backp...|
|B002CMM4M6|Capezio Women's 1...|
|B00HNO32BU|Timberland PRO Me...|
|B00C6BQQW2|New Balance Men's...|
|B00M0NREE0|New Balance Women...|
|B00ZC4VIKK|Donalworld Women ...|
|B00291CGH8|Justin Boots Wome...|
|B000P6GKOO|Nike Men's Rosher...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
products_df.drop_duplicates()
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CJL5ES|L.B. Evans Men's ...|
|B002MCVHUQ|New Balance Insol...|
|B00UXBFW6S|Kenox Sling Backp...|
|B002CMM4M6|Capezio Women's 1...|
|B00HNO32BU|Timberland PRO Me...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781| 2015-08-31|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853| 2015-08-31|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660| 2015-08-31|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263| 2015-08-31|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171| 2015-08-31|
|R1WXA9JSC2H1U1|   19324665|B011F9E6LI|      14311457| 2015-08-31|
|R12ENYLFGGNWRV|   50073594|B00HAUP1OI|     264821602| 2015-08-31|
|R2R07E5PNXEUO5|   21706057|B00L1RKOKW|     767118055| 2015-08-31|
|R27BA52AKWMWN3|   13708216|B005WA9MSE|     813856438| 2015-08-31|
| RLF8DOID2KD5O|   40542649|B00BEYQI5C|     661491213| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2R07E5PNXEUO5|          4|            0|          0|   N|                Y|
|R27BA52AKWMWN3|          5|            0|          0|   N|                Y|
| RLF8DOID2KD5O|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<> 5432/AWSdata_db"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table1', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table1', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table1', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table1', mode=mode, properties=config)

##  Deliverable-2:  Bias of Vine Reviews 


In [ ]:
filtered_vine_df=vine_df.filter(vine_df["total_votes"]>=20)
filtered_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [ ]:
# check the type of the variables
filtered_vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [ ]:
from pyspark.sql.types import DoubleType

In [ ]:
cleaned_vine_df=filtered_vine_df.withColumn("helpful_vote_rates",(filtered_vine_df["helpful_votes"])/(filtered_vine_df["total_votes"]))
cleaned_vine_df=cleaned_vine_df.filter(cleaned_vine_df["helpful_vote_rates"]>0.5)
cleaned_vine_df.select(["helpful_vote_rates"]).describe().show()

+-------+-------------------+
|summary| helpful_vote_rates|
+-------+-------------------+
|  count|              26993|
|   mean| 0.9195626317197761|
| stddev|0.07500562785980776|
|    min| 0.5102040816326531|
|    max|                1.0|
+-------+-------------------+



In [ ]:
# show the filtered dara
cleaned_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_rates|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|0.9183673469387755|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|               1.0|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|0.5925925925925926|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|0.9047619047619048|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|0.8636363636363636|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [ ]:
# Alternative 2: filtering rationof helpfulvotes/total vaotes>0.5. No new column is generated
alt_cleaned_vine_df=filtered_vine_df.filter((filtered_vine_df["helpful_votes"])/(filtered_vine_df["total_votes"])>0.5)
alt_cleaned_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
#  filtering (vine ==Y),
Yes_vine_df=cleaned_vine_df.filter(cleaned_vine_df["vine"]=="Y")
Yes_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_rates|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|0.9545454545454546|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|0.8947368421052632|
|R2MPEQ4SPTEQNS|          4|          180|        184|   Y|                N|0.9782608695652174|
| RIR0D3KJ0CQ31|          4|           21|         21|   Y|                N|               1.0|
|R1SPWJDHUWWC5E|          5|           88|         98|   Y|                N|0.8979591836734694|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [ ]:
#  filtering (vine ==N)
No_vine_df=cleaned_vine_df.filter(cleaned_vine_df["vine"]=="N")
No_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_rates|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|0.9183673469387755|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|               1.0|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|0.5925925925925926|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|0.9047619047619048|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|0.8636363636363636|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows

